---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
from llama_index.core import SimpleDirectoryReader, Settings, StorageContext, VectorStoreIndex
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

import nest_asyncio

nest_asyncio.apply()

In [2]:
documents = SimpleDirectoryReader("../data", recursive=True).load_data() 

base_url = "http://127.0.0.1:11434"
Settings.embed_model = OllamaEmbedding(
    base_url=base_url,
    model_name="nomic-embed-text",
)

# ollama
Settings.llm = Ollama(model="llama3", request_timeout=360.0)

# client = qdrant_client.QdrantClient(url="http://localhost:6333")
aclient = qdrant_client.AsyncQdrantClient(host="localhost", port=6333)
vector_store = QdrantVectorStore(aclient=aclient, collection_name="my_collection")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

splitter = SemanticSplitterNodeParser(embed_model = Settings.embed_model)

```powershell
docker run -p 6333:6333 -p 6334:6334 `
     -v ${pwd}\qdrant_storage:/qdrant/storage:z `
     qdrant/qdrant
```

In [3]:
# Settings.chunk_size = 512

In [3]:
# Раздиление документов на ноды
nodes = splitter.get_nodes_from_documents(documents)

In [4]:
# Добавление в qdrant
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context, use_async=True)

--------------------------------

In [5]:
query_engine = index.as_query_engine()

In [29]:
# question = "Расскажи о документе, кратко в виде списка:"
# plus_context = f"Отвечай на русском: {question}"
# response = await query_engine.aquery(plus_context)

In [30]:
# for i in str(response).split("\n"):
#     print(i)

Документ содержит следующие информацию:

* Авторы исследования: Терри Юэ Жуо, Юйjin Хуан, Чуньян Чен и Зеньчжан Син.
* Название исследования: "Exploring AI Ethics of ChatGPT: A Diagnostic Analysis".
* Источник документа: arXiv preprint arXiv:2301.12867, 2023.
* Содержание документа:
	+ Описание результатов исследований поChatGPT;
	+ Фигуры 28 и 29, демонстрирующие эффективность модели в выполнении задач.


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
from pydantic import BaseModel
from fastapi import FastAPI
import uvicorn

app = FastAPI()

# # Данные для API
# data = [
#     {"id": 1, "name": "Иван", "age": 25},
#     {"id": 2, "name": "Мария", "age": 30},
#     {"id": 3, "name": "Петр", "age": 35},
# ]


# # Модель данных
# class DataItem(BaseModel):
#     id: int
#     name: str
#     age: int


# # Маршрут для получения всех данных
# @app.get("/api/data", response_model=List[DataItem])
# async def get_data():
#     return await asyncio.to_thread(lambda: data)

class InputData(BaseModel):
    question: str


# Маршрут для получения данных по ID
@app.post("/api/llm/")
async def put_llm_ans(data: InputData):
    plus_context = f"Отвечай на русском: {data.question}"
    output = {
        "answer": str(await query_engine.aquery(plus_context))
    }
    return output


# # Маршрут для создания новых данных
# @app.post("/api/data", status_code=201, response_model=DataItem)
# async def create_data(item: DataItem):
#     new_item = item.dict()
#     data.append(new_item)
#     return new_item

In [9]:
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [16616]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     91.236.142.163:0 - "POST /api/llm HTTP/1.1" 307 Temporary Redirect
INFO:     91.236.142.163:0 - "POST /api/llm/ HTTP/1.1" 200 OK
INFO:     194.50.15.82:0 - "POST /api/llm HTTP/1.1" 307 Temporary Redirect
INFO:     194.50.15.82:0 - "POST /api/llm/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16616]


KeyboardInterrupt: 

---------------------------------